In [1]:
# Import libraries
## for profiling
from profiler.main import profile

# for titiler-xarray
from typing import Any
from zarr_reader import xarray_open_dataset, get_variable
import morecantile
from xarray_tile_reader import XarrayTileReader

# for titiler-pgstac
from titiler.pgstac.mosaic import PGSTACBackend
from typing import Any, Dict, List, Tuple
from rio_tiler.mosaic import mosaic_reader
from rio_tiler.models import ImageData
from geojson_pydantic import Polygon

# Only if wanting to see a picture
import io
from PIL import Image

In [5]:
import os
os.environ['AWS_PROFILE'] = 'veda-dev'

In [4]:
xyz_tile = (0,0,0)

## Profile titiler-pgstac

In [ ]:
%load_ext autoreload
%autoreload
import pgstac.profile_pgstac as profile_pgstac 


In [ ]:
query = {
  "collections": [
    "CMIP6_ensemble_median_TAS"
  ],
  "filter": {
    "op": "t_intersects",
    "args": [
      {
        "property": "datetime"
      },
      {
        "interval": [
          "1950-04-01T00:00:00Z",
          #"1950-05-01T00:00:00Z"
        ]
      }
    ]
  },
  "filter-lang": "cql2-json"
}

image_and_assets, logs = profile_pgstac.tile(*xyz_tile, query=dict(query))
logs

## Profile titiler-xarray

In [2]:
@profile(add_to_return=True, quiet=False, log_library='s3fs')
def tile(src_path: str, x: int, y: int, z: int, *, variable: str, time_slice: str = None, **kwargs: Any):

    with xarray_open_dataset(
        src_path,
        group=z,
        decode_times=False,
        **kwargs,
    ) as dataset:
        dataarray = get_variable(dataset, variable=variable, time_slice=time_slice)
        
        with XarrayTileReader(dataarray) as src_dst:
            return src_dst.tile(x, y, z)

In [8]:
image_and_assets, logs = tile(
    "s3://power-analysis-ready-datastore/power_901_monthly_meteorology_utc.zarr", 
    *xyz_tile,
    reference=False,
    variable="TS",
)
# logs

Time elapsed for xarray.open_datset: 910.99381
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/.zmetadata'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/lat/0'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/lon/0'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/0'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/1'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/10'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/11'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key'

AttributeError: 'Dataset' object has no attribute 'rio'

In [ ]:
content = image_and_assets.render(
    img_format='PNG'
)

im = Image.open(io.BytesIO(content))
im